# Nursery problem - Problema de la guardería

En la Europa de los 80's, más específicamente en 

La configuración inicial para trabajar en el entorno está dada por las siguientes líneas de código de importación de librerías y ubicación de nuestro DataSet.

In [46]:
# OS requirements
!pip install -U pandas-profiling

Requirement already up-to-date: pandas-profiling in c:\users\lenovo\documents\anaconda3\lib\site-packages (2.3.0)


In [47]:
#ROOT_DIR = '/content/gdrive/'
#PERSONAL_DIR = ROOT_DIR + 'My Drive/'
#SHARED_UNITS_DIR = ROOT_DIR + 'Shared drives/'
#NURSERY_INFO_DIR = SHARED_UNITS_DIR + 'SimulaciónP/Nursery/'
#from google.colab import drive
#drive.mount(ROOT_DIR, force_remount=True)
# Importacion estandar de la libreria Pandas
# libreria manipulacion de datos
import os
import pandas as pd
import pandas_profiling as pp
import matplotlib.pyplot as plt
import numpy as np
import collections
%matplotlib inline


## Perfilamiento del DataSet
A continuación trataremos de explorar algo de información que pueda sernos de utilidad para abordar el problema con los modelos de ML adecuados.

In [48]:
CHARACTERISTIC_COLS = ['parents', 'has_nurs', 'form', 'children', 'housing', 'finance', 'social', 'health']
FULL_COLS = ['parents', 'has_nurs', 'form', 'children', 'housing', 'finance', 'social', 'health', 'RESULT']
ndf = pd.read_csv('nursery.csv', delimiter=',', sep='\t', index_col=False) # Para que el dataset contenga las características y la variable objetivo
ndf.profile_report(style={'full_width':True})

In [49]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

def oneHotEncoder(data):
    data_encoded = np.zeros((data.shape[0],0))
    if len(data.shape) == 1:
        num_of_cols = 1
    else:
        num_of_cols = data.shape[1]
    for col_index in range(num_of_cols):
        if num_of_cols == 1:
            values = array(data)
        else:
            values = array(data[:,col_index])
        values = values.reshape((-1,1))
        onehot_encoder = OneHotEncoder(sparse=False, categories='auto')
        onehot_encoded = onehot_encoder.fit_transform(values)
        data_encoded = np.concatenate((data_encoded, onehot_encoded), axis = 1)
    return data_encoded

In [50]:
ndf = ndf[ndf['RESULT'] != 'recommend'] # Se remueve del conjunto de datos la categoría que solo tiene 2 muestras.
X_original = ndf.iloc[:, :-1].values
Y_original = ndf.iloc[:, 8].values
X_encoded = oneHotEncoder(X_original)
Y_encoded = oneHotEncoder(Y_original)

In [51]:
rows_length, cols_length = ndf.shape
print('Muestras:',rows_length, '- Variables/Atributos:', cols_length)

Muestras: 12958 - Variables/Atributos: 9


In [56]:
############## Plantilla de partición de datos ##################
from sklearn.model_selection import StratifiedKFold
X_train, X_test, Y_train, Y_test
skf = StratifiedKFold(n_splits = 3, shuffle = True)
for train_index, test_index in skf.split(X_encoded, Y_original):
    X_train, X_test = X_encoded[train_index], X_encoded[test_index]
    Y_train, Y_test = Y_original[train_index], Y_original[test_index]
############## Plantilla de partición de datos ##################

In [60]:
from sklearn.model_selection import StratifiedKFold
# Ajustar el modelo
from sklearn.naive_bayes import GaussianNB

def naive_bayes(x_train, x_test, y_train, y_test):
    
    Folds = 4
    EficienciaTrain = np.zeros(Folds)
    EficienciaVal = np.zeros(Folds)
    
    classifier = GaussianNB()
    classifier.fit(x_train, y_train)
    
    # Predecir los datos de prueba
    y_pred_train = classifier.predict(x_train)
    # Predecir los datos de prueba
    y_pred = classifier.predict(x_test)
    
    #Evaluamos las predicciones del modelo con los datos de test
    EficienciaTrain = np.mean(y_pred_train == y_train)
    EficienciaVal = np.mean(y_pred == y_test)
    
    print('Eficiencia durante el entrenamiento = ' + str(np.mean(EficienciaTrain)) + '+-' + str(np.std(EficienciaTrain)))
    print('Eficiencia durante la validación = ' + str(np.mean(EficienciaVal)) + '+-' + str(np.std(EficienciaVal)))

    return y_pred

In [61]:
naive_bayes(X_train, X_test, Y_train, Y_test)

Eficiencia durante el entrenamiento = 0.8435004051394838+-0.0
Eficiencia durante la validación = 0.8451030331095161+-0.0


array(['priority', 'not_recom', 'priority', ..., 'spec_prior',
       'spec_prior', 'spec_prior'], dtype='<U10')